In [1]:
import os
import glob
import pandas as pd
import matplotlib
import numpy as np
from numpy import NaN
import matplotlib.pyplot as plt

In [2]:
# Combine all the CSV files
path = r'../data'
all_csv = glob.glob(path + "/*.csv")

temp = []
for csv in all_csv:
    df = pd.read_csv(csv, index_col=0, header=0, thousands=',')
    temp.append(df)

df = pd.concat(temp, axis=0, ignore_index=True)
df.head()

,Service,Type,Rating,Hires,Price,Zip Code
0,"Katie Beth Nutrition, Registered Dietitian",12,5.0,67.0,495.0,36104
1,Cassi Nunes,12,5.0,164.0,249.0,36104
2,Beau Gardner,12,5.0,142.0,100.0,36104
3,Nolan King's Nutrition and Training,12,5.0,744.0,199.0,36104
4,Weightloss For Busy Proffessionals,12,5.0,NaN,NaN,36104


In [3]:
df.shape

(27694, 6)

In [4]:
# Check and fix dtype
df.dtypes

Service      object
Type          int64
Rating      float64
Hires       float64
Price        object
Zip Code      int64
dtype: object

In [5]:
# See some prices values
df['Price'].unique()[:19]

# Need to fix that hour thing

array([495.0, 249.0, 100.0, 199.0, nan, 170.0, 57.0, 125.0, 99.0, 35.0,
       175.0, 75.0, 80.0, 300.0, 200.0, 40.0, 85.0, 79.0, 120.0],
      dtype=object)

In [6]:
# See different types of payment
df['Price'] = df['Price'].astype(str)

tempList = []
checkUnique = []
for i in df["Price"]:
    x = i.split('/')
    tempList.append(x)
    
for i in tempList:
    try:
        x = i[1]
        checkUnique.append(x)
    except:
        pass

checkUnique = list(dict.fromkeys(checkUnique))
checkUnique

['on-site', 'session', 'logo', 'hour', 'walk', 'cat', 'visit']

In [7]:
# Removed the hour into another column
tempPrice = []
tempPriceType = []
for i in df['Price']:
    x = i.split('/')
    tempPrice.append(x[0])
    try:
        y = x[1]
    except:
        y = ""
    tempPriceType.append(y)

# Add it to dataframe and check
df['Price'] = tempPrice
df['Price Type'] = tempPriceType
df['Price Type'] = df['Price Type'].astype(str)

# Convert price column to float
df['Price'] = df['Price'].str.replace(",","").astype(float)

In [8]:
# Double check data types
df.dtypes

Service        object
Type            int64
Rating        float64
Hires         float64
Price         float64
Zip Code        int64
Price Type     object
dtype: object

In [9]:
# Add the service type from database
serviceDf = pd.read_csv('../src/all_services.csv')

dfMerge = df.rename(columns={"Type": "ID"})
dfMerge = pd.merge(dfMerge, serviceDf, left_on='ID', right_on='ID',
                   how='left').drop('ID', axis=1)
df = dfMerge

In [10]:
# See shape
df.shape

(27694, 9)

In [11]:
# Check for duplicates
dfDuplicate = df[df.duplicated()]
dfDuplicate

# Deal with duplicates accordingly if there are any

,Service,Rating,Hires,Price,Zip Code,Price Type,Services,Type,Segments
9542,Jnana Massage Therapy,5.0,NaN,80.0,85001,,Massage Therapy,Main,Wellness


In [12]:
# See overview of data
df.describe()

,Rating,Hires,Price,Zip Code
count,26395.000000,19778.000000,19582.000000,27694.000000
mean,4.934836,67.960006,139.393065,49793.858923
std,0.227697,144.892384,140.250243,31619.250426
min,1.000000,2.000000,1.000000,2201.000000
25%,5.000000,8.000000,62.000000,21202.000000
50%,5.000000,22.000000,100.000000,50309.000000
75%,5.000000,59.000000,170.000000,78701.000000
max,5.000000,2041.000000,6000.000000,99801.000000


In [13]:
# See basic count
servicesCount = df.groupby("Services")["Services"].count().to_frame(
    name="Count").reset_index()
servicesCount = servicesCount.sort_values(by="Count", ascending=False)
servicesCount[
    "Contribution"] = servicesCount["Count"] / servicesCount["Count"].sum()
servicesCount

,Services,Count,Contribution
22,Statistical Data Analysis,2730,0.098577
13,Life Coaching,2730,0.098577
2,Business Tax Preparation,2730,0.098577
17,Personal Training,2730,0.098577
4,Computer Repair,2730,0.098577
16,Nutritionist,2730,0.098577
14,Logo Design,2730,0.098577
0,Accounting,1372,0.049541
21,Roof Repair or Maintenance,1147,0.041417
11,Interior Design,993,0.035856


In [14]:
# Get implied revenue
df["Implied Revenue"] = df["Price"] * df["Hires"]

# Get sum of implied revenue by segment
dfRevenuBySegment = df.groupby("Segments")["Implied Revenue"].sum().to_frame(
    name="Implied Revenue").reset_index()
dfRevenuBySegment["Contribution"] = dfRevenuBySegment[
    "Implied Revenue"] / dfRevenuBySegment["Implied Revenue"].sum()

# Get sum of implied revenue by services
dfRevenuByServices = df.groupby("Services")["Implied Revenue"].sum().to_frame(
    name="Implied Revenue").reset_index()
dfRevenuByServices["Contribution"] = dfRevenuByServices[
    "Implied Revenue"] / dfRevenuByServices["Implied Revenue"].sum()

# Get sum of implied revenue by zipcodes
dfRevenuByRevenue = df.groupby("Zip Code")["Implied Revenue"].sum().to_frame(
    name="Implied Revenue").reset_index()
dfRevenuByRevenue["Contribution"] = dfRevenuByRevenue[
    "Implied Revenue"] / dfRevenuByRevenue["Implied Revenue"].sum()

In [15]:
dfRevenuBySegment.sort_values(by="Contribution", ascending=False)

,Segments,Implied Revenue,Contribution
0,Business,131620933.0,0.641843
4,Wellness,64214887.0,0.313141
2,Home Improvement,7152116.0,0.034877
1,Events,1539953.0,0.007510
3,Pets,539208.0,0.002629


In [16]:
dfRevenuByServices.sort_values(by="Contribution", ascending=False)

,Services,Implied Revenue,Contribution
14,Logo Design,83128150.0,0.405370
2,Business Tax Preparation,32240677.0,0.157220
16,Nutritionist,31362101.0,0.152936
17,Personal Training,20585865.0,0.100386
4,Computer Repair,13555566.0,0.066103
13,Life Coaching,8646424.0,0.042164
15,Massage Therapy,3042009.0,0.014834
10,House Cleaning,2896900.0,0.014127
0,Accounting,2157469.0,0.010521
9,Handyman,1702886.0,0.008304


In [17]:
dfRevenuByRevenue.sort_values(by="Contribution", ascending=False)

,Zip Code,Implied Revenue,Contribution
13,10007,3362970.0,0.016399
14,11021,3080153.0,0.015020
11,7102,2751798.0,0.013419
20,20001,2674891.0,0.013044
79,90013,2609801.0,0.012727
...,...,...,...
88,98507,2029100.0,0.009895
63,70802,2029098.0,0.009895
50,57501,2006361.0,0.009784
72,83702,2001235.0,0.009759


In [18]:
# Rearrange columsn
df = df[[
    'Segments', 'Services', 'Service', 'Type', 'Hires', 'Price', 'Price Type',
    'Rating', 'Zip Code', 'Implied Revenue'
]]

In [19]:
# See columns
print(df.columns.values)

['Segments' 'Services' 'Service' 'Type' 'Hires' 'Price' 'Price Type'
 'Rating' 'Zip Code' 'Implied Revenue']


In [20]:
df.to_csv("../output/all_services.csv")